In [41]:
import librosa # Biblioteca para leitura de áudio
from IPython.display import Audio
import numpy as np
from scipy.fftpack import dct
import scipy as sp
import matplotlib.pyplot as plt
import csv
from pydub import AudioSegment
import numpy as np
import pandas as pd

In [2]:
!pip install pydub


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [81]:
# Função responsável por segmentar a música em intervalos de acordo com dataSet
def segment_audio(audio, intervals, music_name):
  for (start, end) in intervals:
    segment = audio[start*1000:end*1000]
    segment.export(f"{music_name}/{music_name}[{start},{end}].wav", format="wav")

# Função responsável por criar um data frame para todas as músicas dos dados
# Esse data frame terá a nota correspondente e suas frequencias normalizadas do DCT
def create_df_dct_audios(names_music,all_data_audios, data_set_musics):
    all_df = []

    # Para cada arquivo de musica
    for music_name in names_music:
        # Pega seus dados e suas notas
        result_data = [music for music in all_data_audios if music['name'] == music_name]
        result_label = [music for music in data_set_musics if music['name'] == music_name]
        # Aplica no dct e salva no dataframe
        dct_data = []
        for audio in result_data[0]['data']:
            dct_data.append(abs(sp.fft.dct(audio, norm='ortho'))[:12])
        df = pd.DataFrame({
            'Label': result_label[0]['achord_segments'],
            'DCT': dct_data
        })
        all_df.append(df)
        
    
    return all_df

In [83]:
# Leitura do nome das músicas
file_names_musics = 'list_musics.txt'
names_musics = []
with open(file_names_musics, mode='r') as file:
    for line in file:
        if line == '\n':
            continue
        line = line.strip()
        names_musics.append(line)

data_set_musics = []
# Criação de um dicionario contendo as informações do tipo
# Nome, segmentos de audio e acordes para cada segmento
for music_name in names_musics:
    file_audio_segments = f'{music_name}/intervalos_notas.txt'
    time_segments = [] # segmentos de áudio importante da música para criação do dataSet
    achord_segments = [] #achords segments
    with open(file_audio_segments, mode='r') as file:
        for line in file:
            parts = line.strip().split()
            start_time = float(parts[0])
            end_time = float(parts[1])
            label = parts[2]
            if label == 'N':
              continue
            time_segments.append((float(start_time), float(end_time)))
            achord_segments.append(label)
    dic_current_music = {
        'name': music_name,
        'time_segments': time_segments,
        'achord_segments': achord_segments
    }
    data_set_musics.append(dic_current_music)


In [84]:
# Captura os audios originais de cada musica e segmenta de acordo com os intervalos de tempo
all_original_audios = []
for music_name in names_musics:
    audio_path = f'{music_name}/{music_name}.wav'
    audio_original = AudioSegment.from_file(audio_path)
    result = [music for music in data_set_musics if music['name'] == music_name]
    time_segments = result[0]['time_segments']
    segment_audio(audio_original, time_segments, music_name)

In [88]:
all_data_audios = []
# Para cada arquivo de musica, segmenta o audio das musicas originais em intervalos dado pelo data set
for music_name in names_musics:
    result = [music for music in data_set_musics if music['name'] == music_name]
    time_segments = result[0]['time_segments']
    data_audio = []
    # Divisão da musica em intervalos e para cada intervalo armazena seus dados
    for (start, end) in time_segments:
        audio_path = f"{music_name}/{music_name}[{start},{end}].wav"
        y, sr = librosa.load(audio_path, sr=None)  # 'y' é o array com os dados de áudio, 'sr' é a taxa de amostragem
        data_audio.append(y)
    # Cria um dicionario contendo o nome da musica e um vetor bidimensional, onde para cada intervalo vamos ter um array de dados de audio para aquele intervalo
    dic_current_music = {
        'name': music_name,
        'data': data_audio
        }    
    all_data_audios.append(dic_current_music)

all_df_per_music = create_df_dct_audios(names_musics, all_data_audios, data_set_musics)


       Label                                                DCT
0          E  [0.14012113, 0.14712815, 0.14936303, 0.0172292...
1          A  [0.013855663, 0.03156995, 0.026350904, 0.12878...
2          E  [0.13060692, 0.05941716, 0.078089595, 0.036042...
3          B  [0.046385247, 0.00016182648, 0.028308276, 0.03...
4          E  [0.11023276, 0.14720166, 0.17763585, 0.2148806...
5      E:7/3  [0.039325707, 0.059402987, 0.12769395, 0.19411...
6          A  [0.056254942, 0.012674196, 0.025040122, 0.0324...
7   A:min/b3  [0.07840788, 0.022058368, 0.08521865, 0.016518...
8          E  [0.009787019, 0.1343666, 0.035948515, 0.032844...
9          B  [0.07034565, 0.09474815, 0.004579872, 0.120591...
10         E  [0.057095475, 0.039320014, 0.028444944, 0.0350...
11         A  [0.050989646, 0.07264544, 0.07267993, 0.039554...
